In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
import os
import random

# Define the CNN model

In [9]:
"""
    Split the dataset into training and testing sets.

    Args:
    - directory: The directory containing the dataset.
    - ratio: The ratio of training data to total data (default is 0.8).

    Returns:
    - train_files: List of file paths for training.
    - test_files: List of file paths for testing.
    """
def split_dataset(directory, ratio=0.8):
    
    # Get all file paths in the directory
    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                file_path = os.path.join(root, file)
                all_files.append(file_path)

    # Shuffle the file paths
    random.shuffle(all_files)

    # Split the file paths into training and testing sets
    split_idx = int(len(all_files) * ratio)
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]

    return train_files, test_files

In [7]:
num_classes = 411
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output to feed into a dense layer
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

# Output layer
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 239, 319, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 118, 158, 64)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 116, 156, 128)     73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 58, 78, 128)     

In [10]:
directory = "Processed-CASIA-Iris-Interval"
train_files, test_files = split_dataset(directory, ratio=0.7)

print("Number of training files:", len(train_files))
print("Number of testing files:", len(test_files))

Number of training files: 1847
Number of testing files: 792


In [13]:
# Define image dimensions
img_width, img_height = 640, 480
input_shape = (img_height, img_width, 1)  # Grayscale image

# Define batch size
batch_size = 32

# Define number of classes
num_classes = 2  # Assuming binary classification

# Define data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_list(
    train_files,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

# Define data generator for testing data
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    preprocessing_function=preprocess_input
)

test_generator = test_datagen.flow_from_list(
    test_files,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

AttributeError: 'ImageDataGenerator' object has no attribute 'flow_from_list'

In [12]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test accuracy:", accuracy)

NameError: name 'train_generator' is not defined